In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
import pickle

In [10]:
# read datasets
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# process columns, apply LabelEncoder to categorical features
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

Shape train: (4209, 378)
Shape test: (4209, 377)


In [11]:
from sklearn.decomposition import PCA, FastICA
n_comp = 10

# PCA
pca = PCA(n_components=n_comp, random_state=42)
pca2_results_train = pca.fit_transform(train.drop(["y"], axis=1))
pca2_results_test = pca.transform(test)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(train.drop(["y"], axis=1))
ica2_results_test = ica.transform(test)

# Append decomposition components to datasets
for i in range(1, n_comp+1):
    train['pca_' + str(i)] = pca2_results_train[:,i-1]
    test['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    train['ica_' + str(i)] = ica2_results_train[:,i-1]
    test['ica_' + str(i)] = ica2_results_test[:, i-1]
    
y_train = train["y"]
y_mean = np.mean(y_train)

In [13]:
import xgboost as xgb

# prepare dict of params for xgboost to run with
xgb_params = {
    'n_trees': 500, 
    'eta': 0.005,
    'max_depth': 4,
    'subsample': 0.95,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

# form DMatrices for Xgboost training
dtrain = xgb.DMatrix(train.drop('y', axis=1), y_train)
dtest = xgb.DMatrix(test)

# xgboost, cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=800, # increase to have better results (~700)
                   early_stopping_rounds=50,
                   verbose_eval=50, 
                   show_stdv=False
                  )

num_boost_rounds = len(cv_result)
print(num_boost_rounds)

# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)
pickle.dump( model, open( "bench_mark_boost.p", "wb" ) )

[0]	train-rmse:12.6399	test-rmse:12.6383
[50]	train-rmse:11.0903	test-rmse:11.1515
[100]	train-rmse:10.0181	test-rmse:10.1468
[150]	train-rmse:9.28968	test-rmse:9.48773
[200]	train-rmse:8.80275	test-rmse:9.06668
[250]	train-rmse:8.47846	test-rmse:8.80237
[300]	train-rmse:8.26051	test-rmse:8.63724
[350]	train-rmse:8.09259	test-rmse:8.53683
[400]	train-rmse:7.94862	test-rmse:8.47724
[450]	train-rmse:7.83335	test-rmse:8.44244
[500]	train-rmse:7.7256	test-rmse:8.42322
[550]	train-rmse:7.63521	test-rmse:8.41266
[600]	train-rmse:7.5545	test-rmse:8.40571
[650]	train-rmse:7.48302	test-rmse:8.40385
[700]	train-rmse:7.41281	test-rmse:8.40457
673


In [ ]:
# check f2-score (to get higher score - increase num_boost_round in previous cell)
from sklearn.metrics import r2_score, mean_squared_error

# now fixed, correct calculation
print(r2_score(dtrain.get_label(), model.predict(dtrain)))
print(mean_squared_error(dtrain.get_label(), model.predict(dtrain)))

In [ ]:
# make predictions and save results
y_pred = model.predict(dtest)
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': y_pred})
output.to_csv('xgboost-depth{}-pca-ica.csv'.format(xgb_params['max_depth']), index=False)